## Introduction to Crowd

*Crowd* is a social network simulation framework which simplifies and fastens the process of developing agent-based models and simulations on networks. In this notebook, we walk through the steps of developing a basic simulation with Crowd, using an SIR (Susceptible-Infected-Recovered) model, commonly utilized as a base model in epidemiological studies.

#### 1. Create or load a project

In Crowd, simulation settings, datasets and results are stored in a Project structure. As the first step, we import the Project class from the *project_management* module.

In [4]:
from crowd.project_management.project import Project

To start defining a simulation, we can either create a new project or load an existing one. Creating a project requires entering a name, a date, and a quick summary about the project's topic. 

Last parameter *node* denotes that this is a simulation where we are interested in the changes of nodes. The other option is *edge*, which denotes simulations where edges are modified. 

In [ ]:
project_name = "simplediffusion"
creation_date = "19/10/2024"
info = "Diffusion of a virus on a random network"

my_project = Project()

# Create a new project
my_project.create_project(project_name, creation_date, info, "node")

# OR load previous project 
#my_project.load_project(project_name)

Creating random regular graph
Returning G--> Graph with 100 nodes and 200 edges
TO-DO: is it possible to give user more options


Alternatively, we can use Crowd's GUI to configure and run our simulations. The project creation screen is provided below.

<img src="./images/project_creation.png" alt="project_creation">

#### 2. Modify configuration

Crowd employs a configuration file approach to define the simulation settings. This can be done by either modifying the YAML file directly, or using the buttons and selectors in the GUI. 

We will first go over how to construct the YAML file. After the creation of a project, *conf.yaml* file is added automatically on the Project folder. That file itself can be modified, or contents of another file can be copied. We have *sir.yaml* file located in the same directory with this Jupyter notebook, which we will copy its contents, using the following code:

In [ ]:
import os

conf_path = os.path.join(os.path.dirname(__file__), 'sir.yaml')
my_project.update_conf_with_path(conf_path)

Now, we inspect the contents of the configuration file:

```yaml
name: SIR-example
structure:
  random:
    count: 100
    degree: 4
    type: random-regular
definitions:
  pd-model:
    name: diffusion
    nodetypes:
      Susceptible:
        random-with-weight:
          initial-weight: 0.9
      Infected:
        random-with-weight:
          initial-weight: 0.1
      Recovered:
        random-with-weight:
          initial-weight: 0
    node-parameters:
      numerical:
        age:
          - 0
          - 100
    compartments:
      c1:
        ratio: 0.1
        triggering_status: Infected
        type: node-stochastic
      c2:
        iteration-count: 4
        name: healing
        type: count-down
    rules:
      r1:
        - Susceptible
        - Infected
        - c1
      r2:
        - Infected
        - Recovered
        - c2

<b>Explanation of each property:</b>

*name*: This name can later be used in GUI to search for this simulation, so it is useful to pick a meaningful name.

*structure*: Describes how the network will be initialized. Crowd supports reading networks from csv and edgelist files, while providing various network generators from NetworkX and igraph-python libraries. In this example, we choose to generate a random regular network (type), with 100 nodes (count) and each node having a constant degree of 4 (degree).

*definitions*: Holds properties for the network initialization and (optionally) the simulation logic (compartments and rules).

*pd-model*: Means we want to use a predefined model in our simulation. We do not include this keyword for custom simulations.

*name: diffusion*: Specifying that we want to use DiffusionNetwork model of Crowd.

*nodetypes*: Defining each node type for our model and their initialization methods. For this example, we choose (0.9 * node_count) nodes as Susceptible initially. 

*node-parameters*: Can be numerical or categorical. We define each parameter by giving a name. For numerical parameters, Crowd assigns each node a value between [lower-bound, upper-bound] provided. For categorical parameters, a list of possible categories should be given. Alternatively, users can specify a file path to read the values from.

*compartments*: Rule-bits that we use to specify the conditions of a node state change. This feature is based on <a href="https://ndlib.readthedocs.io/en/latest/custom/custom.html">Network Diffusion Library (NDLib)</a> and is integrated into only DiffusionNetwork model. More information about each compartment type can be found on the given link.

*rules*: Consists of: (a) current state of the node, (b) state to move, (c) compartment to use.

<b>Explanation of this simulation's rules:</b> 

*r1:* If a node is susceptible, execute c1. If this
node has any Infected neighbors (triggering status), with a
probability of 0.1 (ratio), the node switches from Susceptible
to Infected state.

*r2:* describes the Infected to Recovered
sequence, which is described with a countdown compartment.
This means that after 4 iterations, the node will switch states.

Alternatively, we can use Crowd's GUI to set these parameters: 

<img src="./images/simulation-settings.png" alt="simulation-settings" width="300">
<img src="./images/data-source.png" alt="data-source" width="300">

<br>

<img src="./images/add-nodetype.png" alt="add-nodetype" width="300">
<img src="./images/nodetype-list.png" alt="nodetype-list" width="300">

#### 3. Define custom methods and run simulation

In [7]:
# 2. Define the custom methods you wish to run

# returns the percentage of infected nodes in every snapshot
def get_percentage_infected(network):
    print(network.node_count)
    return (network.node_count[1] /network.G.number_of_nodes()) * 100

In [8]:
# 3. Run the simulation
my_project.lib_run_simulation(epochs=50,
                              snapshot_period=5,
                              curr_batch=1,
                              after_iteration_methods=[get_percentage_infected])

{0: 86, 1: 14, 2: 0}
{0: 81, 1: 19, 2: 0}
{0: 75, 1: 25, 2: 0}
{0: 73, 1: 27, 2: 0}
{0: 67, 1: 23, 2: 10}
{0: 65, 1: 21, 2: 14}
{0: 60, 1: 21, 2: 19}
{0: 55, 1: 20, 2: 25}
{0: 50, 1: 23, 2: 27}
{0: 46, 1: 21, 2: 33}
{0: 42, 1: 23, 2: 35}
{0: 38, 1: 22, 2: 40}
{0: 37, 1: 18, 2: 45}
{0: 34, 1: 16, 2: 50}
{0: 34, 1: 12, 2: 54}
{0: 33, 1: 9, 2: 58}
{0: 31, 1: 7, 2: 62}
{0: 31, 1: 6, 2: 63}
{0: 30, 1: 4, 2: 66}
{0: 29, 1: 5, 2: 66}
{0: 29, 1: 4, 2: 67}
{0: 29, 1: 2, 2: 69}
{0: 28, 1: 3, 2: 69}
{0: 28, 1: 2, 2: 70}
{0: 28, 1: 1, 2: 71}
{0: 28, 1: 1, 2: 71}
{0: 27, 1: 2, 2: 71}
{0: 27, 1: 1, 2: 72}
{0: 27, 1: 1, 2: 72}
{0: 27, 1: 1, 2: 72}
{0: 26, 1: 2, 2: 72}
{0: 26, 1: 1, 2: 73}
{0: 25, 1: 2, 2: 73}
{0: 25, 1: 2, 2: 73}
{0: 24, 1: 3, 2: 73}
{0: 24, 1: 2, 2: 74}
{0: 24, 1: 2, 2: 74}
{0: 24, 1: 1, 2: 75}
{0: 24, 1: 1, 2: 75}
{0: 24, 1: 0, 2: 76}
{0: 24, 1: 0, 2: 76}
{0: 24, 1: 0, 2: 76}
{0: 24, 1: 0, 2: 76}
{0: 24, 1: 0, 2: 76}
{0: 24, 1: 0, 2: 76}
{0: 24, 1: 0, 2: 76}
{0: 24, 1: 0, 2: 76}
{0